In [1]:
from dotenv import load_dotenv
import os

from langchain_gigachat.chat_models import GigaChat

load_dotenv()

GIGACHAT_API_KEY = os.getenv("GIGACHAT_CREDENTIALS")
if not GIGACHAT_API_KEY:
    raise ValueError("❌ GIGACHAT_CREDENTIALS не найден в .env")

langchain_llm = GigaChat(
    credentials=GIGACHAT_API_KEY,
    verify_ssl_certs=False,
    model="GigaChat-Max",   
    temperature=0.4,
    max_tokens=2048,
)

print("✅ GigaChat LangChain LLM инициализирован")


✅ GigaChat LangChain LLM инициализирован


In [2]:
from types import SimpleNamespace
from typing import List, Dict

import autogen
from autogen import AssistantAgent, UserProxyAgent, LLMConfig


class GigaChatLangChainClient:
    """
    Минимальный клиент для AG2, который внутри использует langchain_llm (GigaChat).
    """

    def __init__(self, config, **kwargs):
        self.model_name = config.get("model", "giga-chat-langchain")

    def create(self, params):
        """
        AG2 передаёт сюда messages и параметры.
        Мы вызываем langchain_llm и возвращаем объект с .choices[...].message.content.
        """

        # params — словарь, который AutoGen передаёт в клиент.
        # В нём есть ключ "messages" — список сообщений в стиле OpenAI:
        messages: List[Dict[str, str]] = params["messages"]
        # превращаем сообщения в один текстовый промпт, так как наш langchain_llm в текущей конфигурации ожидает одну строку
        prompt = "\n".join(f"{m['role']}: {m['content']}" for m in messages)

        # вызываем реальную модель
        text = langchain_llm.predict(prompt)

        # упаковываем результат в формат, похожий на OpenAI
        response = SimpleNamespace()
        response.model = self.model_name
        response.choices = []

        choice = SimpleNamespace()
        msg = SimpleNamespace()
        msg.content = text
        msg.function_call = None
        choice.message = msg

        response.choices.append(choice)
        return response

        # Структура:
        # response = {
        # model: "giga-chat-langchain",
        # choices: [
        #     {
        #     message: {
        #         content: "Текст ответа модели",
        #         function_call: None
        #     }
        #     }
        # ]
        # }

    # Этот метод AutoGen использует, чтобы достать текст ответа из объекта response.
    def message_retrieval(self, response):
        return [choice.message.content for choice in response.choices]

    # Этот метод нужен, если надо считать стоимость и кол-во токенов. 0 Чтобы AutoGen не падал
    def cost(self, response) -> float:
        return 0.0

    # Служебный метод для сбора статистики, кол-ва токенов и тд. отдаём пустой словарь - AutoGen этого достаточно.
    @staticmethod
    def get_usage(response):
        return {}


print("✅ GigaChatLangChainClient определён")


✅ GigaChatLangChainClient определён


In [3]:
DATA_DESCRIPTION = """
У тебя есть таблица транзакций с такими колонками:

- transaction_id — идентификатор транзакции (целое число)
- transaction_date — дата (строка/дата)
- transaction_qty — количество товара в транзакции (число)
- store_id — идентификатор магазина
- product_id — идентификатор товара
- unit_price — цена за единицу (может быть с копейками)
- product_category — категория товара (например, Coffee, Tea, Bakery)
- product_type — тип товара (например, Brewed coffee, Chai tea, Hot chocolate)
- product_detail — конкретный SKU / название позиции (например, Ethiopia Rg, Latte Rg)

Данные лежат в файле 'transactions.xlsx' в текущей директории.
"""
print("✅ Описание данных подготовлено")


✅ Описание данных подготовлено


In [4]:
# LLMConfig — это обёртка/класс из autogen, который хранит настройки для LLM (модели), с которой будет работать агент.
llm_config = LLMConfig({
    "model": "giga-chat-langchain",
    "model_client_cls": "GigaChatLangChainClient",
})

# агенты (AssistantAgent) не работают напрямую с GigaChat, OpenAI, Llama и т.д.;

# вместо этого им дают объект llm_config, где написано:
# какую модель использовать ("model": "...")
# какой клиент обслуживает эту модель ("model_client_cls": "...")


In [5]:
planner_system_message = f"""
Ты аналитик по данным в продуктовой команде.

{DATA_DESCRIPTION}

Твоя задача — по текстовому запросу пользователя:

1. Формализовать требования к выборке/отчёту.
2. Ясно перечислить, какие метрики/поля должны быть в финальном pandas DataFrame.
3. Описать формулы вычисления метрик (словами или псевдокодом).
4. Описать группировки / фильтры / сортировки.
5. Если важны ограничения по датам, обязательно зафиксируй период.

Формат ответа:
- Кратко переформулируй задачу.
- Явно перечисли шаги подготовки данных.
- Явно опиши структуру целевого DataFrame в виде списка столбцов с пояснениями, например:

result_df:
  - store_id: идентификатор магазина
  - total_revenue: суммарная выручка = sum(transaction_qty * unit_price)
  - ...

Не пиши Python-код. Пиши только постановку задачи и формальное описание шага за шагом.
"""

planner = AssistantAgent(
    name="planner",
    system_message=planner_system_message,
    llm_config=llm_config,
)


[autogen.oai.client: 11-29 16:19:17] {929} INFO - Detected custom model client in config: GigaChatLangChainClient, model client can not be used until register_model_client is called.


In [6]:
coder_system_message = f"""
Ты Python-разработчик, который пишет код для анализа данных с помощью pandas.

{DATA_DESCRIPTION}

Ты получаешь на вход формализованное ТЗ от аналитика (описание шагов и целевого DataFrame).
Твоя задача — написать ПОЛНЫЙ код на Python, который:

- Импортирует необходимые библиотеки (как минимум: import pandas as pd).
- Загружает данные из файла 'transactions.xlsx' (лист по умолчанию).
- Выполняет все фильтры, агрегации и расчёт метрик, которые описал аналитик.
- Создаёт финальный pandas DataFrame с указанной структурой и кладёт его в переменную result_df.
- В конце печатает:
    - result_df.shape
    - первые несколько строк result_df (например, result_df.head())

Формат ответа:
- ТОЛЬКО один блок с кодом в формате markdown:
```python
# твой код
БЕЗ дополнительного текста до или после блока.
Код должен быть запускаемым 'как есть'.
"""

coder = AssistantAgent(
name="coder",
system_message=coder_system_message,
llm_config=llm_config,
)

[autogen.oai.client: 11-29 16:19:20] {929} INFO - Detected custom model client in config: GigaChatLangChainClient, model client can not be used until register_model_client is called.


In [7]:
### 4.3. Исполнитель (executor) — запускает код
# В AutoGen именно UserProxyAgent отвечает за физическое выполнение Python-кода.

executor = UserProxyAgent(
    name="executor",
    human_input_mode="NEVER",
    code_execution_config={
        "work_dir": ".",      # где запускать код
        "use_docker": False,  # можно True, если хочешь изолировать окружение
    },
)

In [8]:
result_system_message = f"""
Ты итоговый аналитик-контролёр.

{DATA_DESCRIPTION}

Ты получаешь на вход:
- исходный запрос пользователя,
- формализованное ТЗ от аналитика,
- код на Python, который был выполнен,
- текстовый вывод исполнения (stdout/ошибки).

Твоя задача — проанализировать ситуацию и вернуть ОДИН из двух вариантов JSON:

1) Если код успешно отработал и из получившихся данных можно сделать выводы:
{{
  "status": "REPORT",
  "report": "здесь связный аналитический отчёт на русском языке по результатам (что получилось)"
}}

2) Если:
   - при выполнении кода была ошибка (например, есть 'Traceback' или 'Error' в выводе),
   - или result_df пустой/подозрительно маленький,
   - или по данным нельзя сделать осмысленных выводов,
то верни:
{{
  "status": "NEED_FIX",
  "feedback": "что именно не так и что нужно скорректировать в коде или ТЗ"
}}

Требования:
- Ответ ДОЛЖЕН быть строго валидным JSON-объектом без лишнего текста и без ``` вокруг.
- Поле status должно быть только 'REPORT' или 'NEED_FIX'.
"""

result_agent = AssistantAgent(
    name="result_agent",
    system_message=result_system_message,
    llm_config=llm_config,
)


[autogen.oai.client: 11-29 16:19:27] {929} INFO - Detected custom model client in config: GigaChatLangChainClient, model client can not be used until register_model_client is called.


In [9]:
# прикрепление кастомного клиента GigaChatLangChainClient к каждому агенту
planner.register_model_client(model_client_cls=GigaChatLangChainClient)
coder.register_model_client(model_client_cls=GigaChatLangChainClient)
result_agent.register_model_client(model_client_cls=GigaChatLangChainClient)

print("✅ Агенты planner, coder, executor, result_agent настроены")


✅ Агенты planner, coder, executor, result_agent настроены


In [10]:
manager_system_message = """
Ты менеджер многоагентной системы аналитики данных.

У тебя есть следующие агенты:
- planner: формализует запрос и описывает целевой DataFrame.
- coder: пишет Python-код по ТЗ.
- executor: выполняет код и возвращает stdout/ошибки.
- result_agent: умеет анализировать получившиеся данные и формировать человеческий отчёт.

Текущее состояние пайплайна тебе передаётся в одном сообщении: 
- исходный запрос пользователя,
- последняя версия ТЗ от planner (может дополняться фидбеком),
- последний сгенерированный код (если есть),
- последний вывод исполнения exec_output (если код запускался),
- последний отчёт или фидбек (если есть).

ТВОЯ задача — решить, что делать дальше и вернуть СТРОГО JSON формата:

{
  "action": "PLAN" | "CODE" | "EXECUTE" | "MAKE_REPORT" | "FINISH",
  "reason": "краткое пояснение русским текстом",
  "feedback_for_planner": "опционально, что нужно скорректировать в ТЗ",
  "feedback_for_coder": "опционально, что нужно скорректировать в коде"
}

Смысл действий:
- PLAN: надо снова вызвать planner (например, если ТЗ кривое).
- CODE: надо вызвать coder, передав ему актуальное ТЗ (учитывая feedback_for_coder).
- EXECUTE: надо вызвать executor на последнем коде.
- MAKE_REPORT: надо вызвать result_agent, чтобы он сделал финальный отчёт.
- FINISH: работа завершена (например, отчёт уже есть или дальше улучшать не имеет смысла).

Учитывай:
- Если кода ещё нет — нет смысла EXECUTE или MAKE_REPORT.
- Если ещё не было выполнения — MAKE_REPORT бесполезен.
- Если exec_output содержит ошибки (Traceback, Error) — вероятно, нужен CODE или PLAN.
- Следи за разумностью количества шагов (снаружи всё равно есть лимит шагов).
"""

manager = AssistantAgent(
    name="manager",
    system_message=manager_system_message,
    llm_config=llm_config,
)

manager.register_model_client(model_client_cls=GigaChatLangChainClient)


[autogen.oai.client: 11-29 16:19:33] {929} INFO - Detected custom model client in config: GigaChatLangChainClient, model client can not be used until register_model_client is called.


In [ ]:
def run_pipeline_with_manager(user_query: str, max_steps: int = 8):
    planner_reply = None
    coder_reply = None
    exec_output = None
    final_report = None

    step = 0

    while step < max_steps:
        step += 1
        print(f"\n=== ШАГ {step} ===")

        # Собираем текущее состояние в один текст для менеджера
        state_text = (
            f"Запрос пользователя:\n{user_query}\n\n"
            f"ТЗ от planner (может быть пустым):\n{planner_reply}\n\n"
            f"Последний сгенерированный код (может быть пустым):\n{coder_reply}\n\n"
            f"Последний вывод исполнения exec_output (может быть пустым):\n{exec_output}\n\n"
            f"Последний итоговый отчёт (может быть пустым):\n{final_report}\n\n"
            "Реши, что делать дальше, и верни JSON согласно инструкции."
        )

        manager_reply = manager.generate_reply(
            messages=[{"role": "user", "content": state_text}]
        )
        print("\n🧭 Ответ manager:\n", manager_reply)

        # Парсим JSON от manager
        try:
            import json
            manager_json = json.loads(manager_reply)
        except Exception as e:
            print("⚠️ Не удалось распарсить JSON от manager, завершаем:", e)
            break

        action = manager_json.get("action")
        fb_planner = manager_json.get("feedback_for_planner") or ""
        fb_coder = manager_json.get("feedback_for_coder") or ""
        print("➡️ Действие:", action)

        if action == "PLAN":
            # Вызов planner с учётом фидбека
            planner_prompt = user_query
            if fb_planner:
                planner_prompt += f"\n\nДополнительные замечания к формализации:\n{fb_planner}\n"
            planner_reply = planner.generate_reply(
                messages=[{"role": "user", "content": planner_prompt}]
            )
            print("\n🧠 Новый ответ planner:\n", planner_reply)

        elif action == "CODE":
            if not planner_reply:
                print("⚠️ Нет ТЗ от planner, сначала нужен PLAN.")
                continue
            coder_prompt = (
                "Ниже формализованное ТЗ от аналитика.\n"
                "Напиши код согласно своим инструкциям.\n\n"
                "=== ТЗ аналитика ===\n"
                f"{planner_reply}\n"
                "=== Конец ТЗ ===\n"
            )
            if fb_coder:
                coder_prompt += f"\nДополнительные замечания к коду:\n{fb_coder}\n"

            coder_reply = coder.generate_reply(
                messages=[{"role": "user", "content": coder_prompt}]
            )
            print("\n💻 Новый код coder:\n", coder_reply)

        elif action == "EXECUTE":
            if not coder_reply:
                print("⚠️ Нет кода для исполнения, сначала нужен CODE.")
                continue
            exec_output = executor.generate_reply(
                messages=[{"role": "user", "content": coder_reply}]
            )
            print("\n▶️ Вывод исполнения exec_output:\n", exec_output)

        elif action == "MAKE_REPORT":
            # используем уже существующий result_agent, но теперь он просто делает отчёт,
            # а не решает, чинить ли код
            if not exec_output:
                print("⚠️ Нет exec_output, пока рано делать отчёт.")
                continue

            result_prompt = (
                "Сформируй аналитический отчёт для пользователя на основе данных ниже.\n\n"
                f"Запрос пользователя:\n{user_query}\n\n"
                f"ТЗ от planner:\n{planner_reply}\n\n"
                f"Код, который выполнялся:\n{coder_reply}\n\n"
                f"Вывод исполнения (stdout/ошибки):\n{exec_output}\n\n"
                "Если есть признаки ошибки исполнения — упомяни это в отчёте."
            )

            final_report = result_agent.generate_reply(
                messages=[{"role": "user", "content": result_prompt}]
            )
            print("\n📊 Итоговый отчёт (сырой текст):\n", final_report)
            # Можно сразу отображать красиво:
            display(Markdown(final_report))

        elif action == "FINISH":
            print("⏹ Manager решил остановиться.")
            break

        else:
            print("⚠️ Неизвестное действие, завершаем.")
            break

    return {
        "planner_reply": planner_reply,
        "coder_reply": coder_reply,
        "exec_output": exec_output,
        "final_report": final_report,
    }


In [16]:
import json
from IPython.display import display, Markdown

user_query = """
Хочу узнать информацию за январь 2023 года:
- выручку (общую),
- средний чек (общий),
- самые продаваемые в категории
"""

result = run_pipeline_with_manager(user_query)



=== ШАГ 1 ===

🧭 Ответ manager:
 {
  "action": "PLAN",
  "reason": "Необходимо сформировать техническое задание для дальнейшей работы."
}
➡️ Действие: PLAN

🧠 Новый ответ planner:
 ### Переформулировка задачи:
Необходимо подготовить отчёт по транзакциям за январь 2023 года, включающий следующие показатели:
- общую выручку,
- общий средний чек,
- список наиболее популярных товаров по каждой товарной категории.

---

### Шаги подготовки данных:
1. Загрузить данные из файла `transactions.xlsx`.
2. Преобразовать поле `transaction_date` в формат даты.
3. Отфильтровать строки, соответствующие январю 2023 года (`transaction_date >= '2023-01-01' and transaction_date < '2023-02-01'`).
4. Рассчитать итоговые значения показателей.

---

### Структура целевого DataFrame:

**Общая информация**
- **total_revenue**: общая выручка за месяц ($= \sum(\text{transaction_qty} \times \text{unit_price})$)
- **average_check**: средний чек ($= \frac{\text{total\_revenue}}{\text{количество уникальных чеков}}$)

{
  "status": "NEED_FIX",
  "feedback": "Полученный результат содержит некорректную структуру таблицы. В частности, общие метрики (выручка и средний чек) объединены с детализированными показателями (топ-продукты). Это затрудняет интерпретацию результатов.\n\nРекомендуемые исправления:\n- Разделите результаты на два отдельных DataFrame: один для общих показателей (выручка, средний чек), другой — для списка топ-продуктов по категориям.\n- Убедитесь, что каждый DataFrame имеет корректное количество строк и столбцов."
}


=== ШАГ 5 ===

🧭 Ответ manager:
 {
  "action": "CODE",
  "reason": "Нужно исправить код, чтобы разделить отчёты на две части: одна для общих показателей, другая — для топ-продуктов.",
  "feedback_for_coder": "Разделите результирующий DataFrame на два отдельных объекта: один для общих показателей (выручка, средний чек), второй — для топ-продуктов по категориям. Не объединяйте эти данные в одну таблицу."
}
➡️ Действие: CODE

💻 Новый код coder:
 ```python
import pandas as pd

# Чтение данных из Excel-файла
df = pd.read_excel('transactions.xlsx')

# Преобразование столбца дат в datetime
df['transaction_date'] = pd.to_datetime(df['transaction_date'])

# Фильтруем данные по январю 2023 года
january_2023_data = df[(df['transaction_date'].dt.year == 2023) & (df['transaction_date'].dt.month == 1)]

# Расчет общей выручки
total_revenue = (january_2023_data['transaction_qty'] * january_2023_data['unit_price']).sum()

# Количество уникальных чеков
unique_transactions_count = len(january_2023_data